## Evaluating Semantic Chunking KB with multiple models for RAG

### Overview

This notebook demonstrates Retrieval-Augmented Generation (RAG) using a predefined ground truth to evaluate the effectiveness of multiple models. We utilize Amazon Nova lite, Nova micro, Claude Haiku and Claude Sonnet for generating responses and FloTorch for evaluating these responses against the ground truth after retrieving information.


#### Load env variables

In [ ]:
import json
with open("variables.json", "r") as f:
    variables = json.load(f)

variables

#### Set AWS Credentials

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")


#### Load prompt.json

prompt.json file includes the following:

* system_prompt
* examples for n shot learning
* user_prompt

In [ ]:
prompt_file_path = './data/prompt.json'
with open(prompt_file_path, 'r') as f:
    prompt = json.load(f)

#### Running the evaluation against multiple models



Inference Models considered - Amazon Nova Lite, Amazon Nova Pro, Claude Haiku 3.5, Claude Sonnet 3.5

In [ ]:
inference_models = ['flotorch/nova-pro', 'flotorch/nova-micro',
                'flotorch/us-anthropic-claude-3-5-haiku','flotorch/anthropic-claude-3-5-sonnet']

#### Experiment Config


* **Rerank Model:** Amazon Rerank
* **N-Shot Prompt:** 1
* **Temperature:** 0.1


In [ ]:
exp_config_data = {
    "temp_retrieval_llm": "0.1",
    "gt_data": variables["s3_ground_truth_path"],
    "rerank_model_id": "amazon.rerank-v1:0",
    "retrieval_service": "bedrock",
    "retrieval_model": "us.amazon.nova-lite-v1:0",
    "aws_region": variables['regionName'],
    "n_shot_prompt_guide_obj": prompt,
    "n_shot_prompts": 1
}

#### Load ground truth data

We utilize FloTorch core's S3StorageProvider and JSONReader to load ground truth data for evaluating the RAG pipeline.

In [ ]:
from flotorch_core.storage.storage_provider_factory import StorageProviderFactory
from flotorch_core.reader.json_reader import JSONReader
from flotorch_rag_utils import Question
gt_data = exp_config_data['gt_data']
storage = StorageProviderFactory.create_storage_provider(gt_data)
gt_data_path = storage.get_path(gt_data)
json_reader = JSONReader(storage)
questions = json_reader.read_as_model(gt_data_path, Question)

#### Initialize Inferencer

Creates and returns an appropriate `Inferencer` instance depending on service and the model 

#### Parameters

- `gateway_enabled`: *(bool)* – Enables FloTorch LLM gateway-based invocation if set to `True`.
- `gateway_url`: *(str)* – URL endpoint for the FloTorch LLM Gateway.
- `gateway_api_key`: *(str)* – API key for authenticating requests to the FloTorch LLM gateway.
- `retrieval_service`: *(str)* – Name of the retrieval service (e.g., bedrock, sagemaker).
- `retrieval_model`: *(str)* – The model to use for inference (e.g., `anthropic.claude-v2`).
- `aws_region`: *(str)* – AWS region for service provisioning (e.g., `us-east-1`).
- `iam_role`: *(str)* – IAM role ARN for SageMaker invocation permissions.
- `n_shot_prompts`: *(int)* – Number of few-shot examples to include in prompt.
- `temp_retrieval_llm`: *(float)* – Temperature setting for the language model.
- `n_shot_prompt_guide_obj`: *(Any)* – Few-shot guide object for prompt engineering.

---

#### Behavior

- If `gateway_enabled` is `True`, connects to the FloTorch LLM Gateway using credentials.
- If disabled, falls back to direct model invocation through supported services like AWS Bedrock or AWS SageMaker.
- Supports dynamic few-shot prompting and custom temperature configuration.

---

#### Outcome

Returns a fully configured `Inferencer` object capable of generating answers or completions for queries using the selected language model.

In [ ]:
from flotorch_core.inferencer.inferencer_provider_factory import InferencerProviderFactory

def initialize_inferencer(inference_model: str, exp_config_data: dict):
    inferencer = InferencerProviderFactory.create_inferencer_provider(
        True,
        "https://qa-gateway.flotorch.cloud/api/openai/v1/",
        "sk_YjhlZjc4ZDYtYjQ5Mi00OGE2LWFiZmItMGFjZTE2NTQwMjBi_YsZXC94WX3rvwNkO2B4lJJy04EDb6C7VhNSASTKyO4Y=",
        exp_config_data.get("retrieval_service"),
        inference_model, 
        exp_config_data.get("aws_region"), 
        " ",
        int(exp_config_data.get("n_shot_prompts", 0)), 
        float(exp_config_data.get("temp_retrieval_llm", 0)), 
        exp_config_data.get("n_shot_prompt_guide_obj")
    )
    return inferencer

#### Execute RAG against all the inference models

Perform the retrieval, reranking, and inference steps using the `flotorch-core` library.

In [ ]:
from flotorch_rag_utils import rag_with_flotorch

rag_response_dict = {}
# The evaluation process duration is dependent on the volume of questions and the number of models bases being evaluated. 
# Larger evaluations require more time, generally around 5-6 minutes.
for inference_model in inference_models:
    print(f"{inference_model} inferece started")
    inferencer = initialize_inferencer(inference_model, exp_config_data)
    responses = rag_with_flotorch(exp_config_data, None, None, inferencer, questions)
    rag_response_dict[inference_model] = responses


In [ ]:
rag_response_dict.keys()

#### Write the results to a JSON file

In [ ]:
import json

filename = f"./results/ragas_evaluation_responses_for_different_models.json"

# Save to JSON with proper formatting
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(rag_response_dict, f, indent=4, ensure_ascii=False)